In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d jillanisofttech/brain-stroke-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/47.2k [00:00<?, ?B/s]
100% 47.2k/47.2k [00:00<00:00, 15.6MB/s]
Archive:  brain-stroke-dataset.zip
  inflating: brain_stroke.csv        


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.spatial import distance

class LocallyAdaptiveKNN:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        y_pred = []
        for test_point in X_test:
            distances = [distance.euclidean(test_point, train_point) for train_point in self.X_train]
            sorted_indices = np.argsort(distances)
            k_nearest_indices = sorted_indices[:self.k]
            k_nearest_labels = self.y_train[k_nearest_indices]
            prediction = max(set(k_nearest_labels), key=list(k_nearest_labels).count)
            y_pred.append(prediction)
        return np.array(y_pred)

# Load the dataset
data = pd.read_csv('/content/brain_stroke.csv')

# Drop rows with missing values
data = data.dropna()

# Convert categorical variables to numeric
label_encoder = LabelEncoder()
data['gender'] = label_encoder.fit_transform(data['gender'])
data['ever_married'] = label_encoder.fit_transform(data['ever_married'])
data['work_type'] = label_encoder.fit_transform(data['work_type'])
data['Residence_type'] = label_encoder.fit_transform(data['Residence_type'])
data['smoking_status'] = label_encoder.fit_transform(data['smoking_status'])

# Split the dataset into features and target variable
X = data.drop(columns=['stroke'])  # Features
y = data['stroke']  # Target variable

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Locally Adaptive k-nearest neighbors model
laknn = LocallyAdaptiveKNN()
laknn.fit(X_train.values, y_train.values)

# Make predictions on the test data
y_pred = laknn.predict(X_test.values)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.9418254764292878
Precision: 0.16666666666666666
Recall: 0.018518518518518517
F1-score: 0.03333333333333333
